In [50]:
import numpy as np
import pandas as pd

In [51]:
df = pd.read_csv('dataset/kidney_stone_data.csv')

In [52]:
df.isnull().sum()

treatment     0
stone_size    0
success       0
dtype: int64

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   treatment   700 non-null    object
 1   stone_size  700 non-null    object
 2   success     700 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 16.5+ KB


In [54]:
df.describe()

,success
count,700.000000
mean,0.802857
std,0.398126
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [55]:
df.head()

,treatment,stone_size,success
0,B,large,1
1,A,large,1
2,A,large,0
3,A,large,1
4,A,large,1


In [56]:
x = pd.DataFrame(df.iloc[:,[0,1]])

In [57]:
y = df['success']

In [58]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [59]:
x_train

,treatment,stone_size
82,B,small
51,B,small
220,B,large
669,A,large
545,A,small
...,...,...
71,B,large
106,B,small
270,A,small
435,B,small


In [60]:
x_test

,treatment,stone_size
158,A,large
500,B,small
396,B,small
155,B,large
321,B,small
...,...,...
24,A,small
218,A,small
431,A,large
281,A,large


In [61]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
trf1 = ColumnTransformer( transformers=[
    ('a',OneHotEncoder(drop='first',sparse=False,handle_unknown='ignore'),[0])
],remainder='passthrough')

In [62]:
from sklearn.preprocessing import OrdinalEncoder
trf2 = ColumnTransformer( transformers=[
    ('b',OrdinalEncoder(categories=[['small','large']] ,unknown_value=-1, handle_unknown='use_encoded_value'),[1])
],remainder='passthrough')

In [63]:
from sklearn.feature_selection import SelectKBest,chi2
trf3 = SelectKBest(score_func=chi2,k=2)

In [64]:
from sklearn.linear_model import LogisticRegression
trf4 = LogisticRegression()

In [69]:
from sklearn.pipeline import Pipeline,make_pipeline
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4)
])

In [70]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('a',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False),
                                                  [0])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('b',
                                                  OrdinalEncoder(categories=[['small',
                                                                              'large']],
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  [1])])),
                ('trf3',
                 SelectKBest(k=2,
                             score_func=<function chi2 at 0x000001AAF6FB9C10>)),
                ('trf4', LogisticRegression())])

In [71]:
from sklearn import set_config
set_config(display='diagram')

In [72]:
y_pred = pipe.predict(x_test)

In [74]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7571428571428571

In [81]:
from sklearn.model_selection import  cross_val_score
cross_val_score(pipe , x_train,y_train,cv=5,scoring='accuracy').mean()

0.8142857142857143

In [82]:
import pickle
pickle.dump(pipe,open('ksd.pkl','wb'))

In [107]:
df

,treatment,stone_size,success
0,B,large,1
1,A,large,1
2,A,large,0
3,A,large,1
4,A,large,1
...,...,...,...
695,B,small,0
696,B,small,1
697,B,small,1
698,A,large,1


In [122]:
np.where(.values==0)

(array([  3,   4,  10,  18,  21,  22,  24,  26,  31,  32,  38,  40,  46,
         51,  58,  60,  64,  68,  71,  75,  90,  93, 101, 105, 106, 107,
        114, 120, 121, 122, 124, 126, 128, 138], dtype=int64),)

In [116]:
x_test.loc[2].values

array(['A', 'large'], dtype=object)

In [123]:
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [124]:
y_test

158    1
500    1
396    1
155    0
321    0
      ..
24     1
218    1
431    1
281    0
110    1
Name: success, Length: 140, dtype: int64

In [125]:
x_test

,treatment,stone_size
158,A,large
500,B,small
396,B,small
155,B,large
321,B,small
...,...,...
24,A,small
218,A,small
431,A,large
281,A,large
